# [모듈 2.1] CodeBuild 프로젝트 생성

- 개발자 가이드
    - [Create a build project (AWS CLI)](https://docs.aws.amazon.com/codebuild/latest/userguide/create-project-cli.html)
    

aws codebuild list-projects --sort-by NAME --sort-order ASCENDING    
aws codebuild batch-get-projects --names iris-build-project    

- 개발자 가이드
    - [Create a build project (AWS CLI)](https://docs.aws.amazon.com/codebuild/latest/userguide/create-project-cli.html)
    

aws codebuild list-projects --sort-by NAME --sort-order ASCENDING    
aws codebuild batch-get-projects --names iris-build-project    

# 1. 환경 설정

In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('./src')

## 1.0 기존의 변수 불러오기

In [2]:
%store -r code_pipeline_train_config_json_path
%store -r sm_pipeline_train_config_json_path

from common_utils import load_json

code_pipeline_train_dict = load_json(code_pipeline_train_config_json_path)
sm_pipeline_train_dict = load_json(sm_pipeline_train_config_json_path)

import json
print("Code Pipeline Series Params: ")
print (json.dumps(code_pipeline_train_dict, indent=2))
print("SageMaker Pipeline Series Params: ")
print (json.dumps(sm_pipeline_train_dict, indent=2))



Code Pipeline Series Params: 
{
  "code_pipeline_role_arn": "arn:aws:iam::057716757052:role/code-pipeline-gsmoon",
  "code_build_service_arn": "arn:aws:iam::057716757052:role/codebuild-gsmoon",
  "project_prefix": "CodePipeline-Train-NCF",
  "region": "us-east-1",
  "account_id": "057716757052",
  "train_code_repo_name": "ncf-train",
  "code_build_project_name": "ncf-training-sm-pipeline",
  "bucket": "sagemaker-us-east-1-057716757052",
  "code_pipeline_name": "ncf-training-code-pipeline",
  "model_package_group_name": "NCF-Model-CodePipeline",
  "branch_name": "master"
}
SageMaker Pipeline Series Params: 
{
  "project_prefix": "SageMaker-Train-NCF",
  "s3_input_data_uri": "s3://sagemaker-us-east-1-057716757052/NCFModel/data",
  "sm_pipeline_name": "ncf-sm-pipeline",
  "training_instance_type": "ml.p3.2xlarge",
  "training_instance_count": 1,
  "ModelApprovalStatus": "PendingManualApproval",
  "inference_image_uri": "763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-inference:1.8.1-

## 1.1. 파라미터 설정

In [3]:
import sagemaker
import boto3

code_repository_name = code_pipeline_train_dict["train_code_repo_name"]


# project_prefix = "Train-NCF"
# code_build_project_name = "ncf-training-sm-pipeline"
# sm_pipeline_name = "ncf-sm-pipeline"
# model_package_group_name = "NCF-Model-CodePipeline"
# inference_image_uri = "763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-inference:1.8.1-gpu-py3"

build_service_role = code_pipeline_train_dict["code_build_service_arn"]
region = code_pipeline_train_dict["region"]
account_id = code_pipeline_train_dict["account_id"]
bucket = code_pipeline_train_dict["bucket"]
model_package_group_name = code_pipeline_train_dict["model_package_group_name"]

project_prefix = code_pipeline_train_dict["project_prefix"]
code_build_project_name = code_pipeline_train_dict["code_build_project_name"]
sm_pipeline_name = sm_pipeline_train_dict["sm_pipeline_name"]


print("build_service_role: ", build_service_role)
print("region: ", region)
print("account_id: ", account_id)
print("bucket: ", bucket)
print("project_prefix: ", project_prefix)
print("code_build_project_name: ", code_build_project_name)
print("sm_pipeline_name: ", sm_pipeline_name)
print("model_package_group_name: ", model_package_group_name)
# print("inference_image_uri: ", inference_image_uri)

build_service_role:  arn:aws:iam::057716757052:role/codebuild-gsmoon
region:  us-east-1
account_id:  057716757052
bucket:  sagemaker-us-east-1-057716757052
project_prefix:  CodePipeline-Train-NCF
code_build_project_name:  ncf-training-sm-pipeline
sm_pipeline_name:  ncf-sm-pipeline
model_package_group_name:  NCF-Model-CodePipeline


# 2. 빌드 프로젝트  JSON 정의
- CodeBuild serviceRole 과 SAGEMAKER_PIPELINE_ROLE_ARN 을 동일한 Role 로 사용함.

```
# SAGEMAKER_PIPELINE_ROLE_ARN
"name": "SAGEMAKER_PIPELINE_ROLE_ARN",
"value": build_service_role,
```

```
# CodeBuild serviceRole
  "serviceRole": build_service_role,  
```

In [4]:
build_dic = {
  "name": code_build_project_name,
  "description": "Hello Build Project",
  "source": {
    "type": "CODEPIPELINE",
    "buildspec": "codebuild-buildspec.yml",
  },
  "artifacts": {
    "type": "CODEPIPELINE",
    "name": code_build_project_name,
  },
  "cache": {
    "type": "NO_CACHE",
  },
  "environment": {
            "type": "LINUX_CONTAINER",
            "image": "aws/codebuild/amazonlinux2-x86_64-standard:3.0",
            "computeType": "BUILD_GENERAL1_SMALL",
            "environmentVariables": [
                {
                    "name": "IMAGE_REPO_NAME",
                    "value": "iris-model",
                    "type": "PLAINTEXT"
                },
                {
                    "name": "IMAGE_TAG",
                    "value": "latest",
                    "type": "PLAINTEXT"
                },
                {
                    "name": "AWS_ACCOUNT_ID",
                    "value": account_id,
                    "type": "PLAINTEXT"
                },
                {
                    "name": "SAGEMAKER_PIPELINE_ROLE_ARN",
                    "value": build_service_role,
                    "type": "PLAINTEXT"
                },        
                {
                    "name": "SAGEMAKER_PIPELINE_NAME",
                    "value": sm_pipeline_name,
                    "type": "PLAINTEXT"
                },                        
                {
                    "name": "AWS_DEFAULT_REGION",
                    "value": region,
                    "type": "PLAINTEXT"
                },
                {
                    "name": "TEMPLATE_BUCKET",
                    "value": bucket,
                    "type": "PLAINTEXT"
                },
                {
                    "name": "code_repository_name",
                    "value": code_repository_name,
                    "type": "PLAINTEXT"
                },                
                {
                    "name": "project_prefix",
                    "value": project_prefix,
                    "type": "PLAINTEXT"
                },
                {
                    "name": "model_package_group_name",
                    "value": model_package_group_name,
                    "type": "PLAINTEXT"
                },                                
                {
                    "name": "TEMPLATE_PREFIX",
                    "value": "codebuild",
                    "type": "PLAINTEXT"
                }
                
            ],
            "privilegedMode": False,
            "imagePullCredentialsType": "CODEBUILD"
        },
  "serviceRole": build_service_role,
  "timeoutInMinutes": 60,
  "queuedTimeoutInMinutes": 480,
  "badgeEnabled": False,
  "logsConfig": {
    "cloudWatchLogs": {
      "status": "ENABLED",
    },
    "s3Logs": {
      "status": "DISABLED",
      "encryptionDisabled": False
    }
  },
}

## 2.1. Dic 포맷을 JSON 으로 변경

In [5]:
import json

build_json = json.dumps(build_dic)
json.loads(build_json)

{'name': 'ncf-training-sm-pipeline',
 'description': 'Hello Build Project',
 'source': {'type': 'CODEPIPELINE', 'buildspec': 'codebuild-buildspec.yml'},
 'artifacts': {'type': 'CODEPIPELINE', 'name': 'ncf-training-sm-pipeline'},
 'cache': {'type': 'NO_CACHE'},
 'environment': {'type': 'LINUX_CONTAINER',
  'image': 'aws/codebuild/amazonlinux2-x86_64-standard:3.0',
  'computeType': 'BUILD_GENERAL1_SMALL',
  'environmentVariables': [{'name': 'IMAGE_REPO_NAME',
    'value': 'iris-model',
    'type': 'PLAINTEXT'},
   {'name': 'IMAGE_TAG', 'value': 'latest', 'type': 'PLAINTEXT'},
   {'name': 'AWS_ACCOUNT_ID', 'value': '057716757052', 'type': 'PLAINTEXT'},
   {'name': 'SAGEMAKER_PIPELINE_ROLE_ARN',
    'value': 'arn:aws:iam::057716757052:role/codebuild-gsmoon',
    'type': 'PLAINTEXT'},
   {'name': 'SAGEMAKER_PIPELINE_NAME',
    'value': 'ncf-sm-pipeline',
    'type': 'PLAINTEXT'},
   {'name': 'AWS_DEFAULT_REGION', 'value': 'us-east-1', 'type': 'PLAINTEXT'},
   {'name': 'TEMPLATE_BUCKET',
   

## 2.2. JSON 파일 저장

In [6]:
json_file_path = 'src/train_code_build.json'
with open(json_file_path, "w") as outfile:
    outfile.write(build_json)

In [7]:
with open(json_file_path) as json_file:
    json_data = json.load(json_file)
    
json_data    

{'name': 'ncf-training-sm-pipeline',
 'description': 'Hello Build Project',
 'source': {'type': 'CODEPIPELINE', 'buildspec': 'codebuild-buildspec.yml'},
 'artifacts': {'type': 'CODEPIPELINE', 'name': 'ncf-training-sm-pipeline'},
 'cache': {'type': 'NO_CACHE'},
 'environment': {'type': 'LINUX_CONTAINER',
  'image': 'aws/codebuild/amazonlinux2-x86_64-standard:3.0',
  'computeType': 'BUILD_GENERAL1_SMALL',
  'environmentVariables': [{'name': 'IMAGE_REPO_NAME',
    'value': 'iris-model',
    'type': 'PLAINTEXT'},
   {'name': 'IMAGE_TAG', 'value': 'latest', 'type': 'PLAINTEXT'},
   {'name': 'AWS_ACCOUNT_ID', 'value': '057716757052', 'type': 'PLAINTEXT'},
   {'name': 'SAGEMAKER_PIPELINE_ROLE_ARN',
    'value': 'arn:aws:iam::057716757052:role/codebuild-gsmoon',
    'type': 'PLAINTEXT'},
   {'name': 'SAGEMAKER_PIPELINE_NAME',
    'value': 'ncf-sm-pipeline',
    'type': 'PLAINTEXT'},
   {'name': 'AWS_DEFAULT_REGION', 'value': 'us-east-1', 'type': 'PLAINTEXT'},
   {'name': 'TEMPLATE_BUCKET',
   

# 3. 빌드 프로젝트 생성

In [8]:
%%sh -s {json_file_path}
json_file_path=$1
echo $json_file_path
aws codebuild create-project --cli-input-json file://$json_file_path

src/train_code_build.json
{
    "project": {
        "name": "ncf-training-sm-pipeline",
        "arn": "arn:aws:codebuild:us-east-1:057716757052:project/ncf-training-sm-pipeline",
        "description": "Hello Build Project",
        "source": {
            "type": "CODEPIPELINE",
            "buildspec": "codebuild-buildspec.yml",
            "insecureSsl": false
        },
        "artifacts": {
            "type": "CODEPIPELINE",
            "name": "ncf-training-sm-pipeline",
            "packaging": "NONE",
            "encryptionDisabled": false
        },
        "cache": {
            "type": "NO_CACHE"
        },
        "environment": {
            "type": "LINUX_CONTAINER",
            "image": "aws/codebuild/amazonlinux2-x86_64-standard:3.0",
            "computeType": "BUILD_GENERAL1_SMALL",
            "environmentVariables": [
                {
                    "name": "IMAGE_REPO_NAME",
                    "value": "iris-model",
                    "type": "PLAINTEX